In [1]:
%matplotlib inline

# Pytorch 全相连网络(Full Connected Network)

In [2]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python

from __future__ import print_function

导入MNIST数据集

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision

# hyper-parameter
training_epoch=100
num_classes = 10
learning_rate=1e-3
batch_size=100

# 查看Pytorch是否支持GPU
GPU_FLAG = torch.cuda.is_available()
print('CUDA available?', GPU_FLAG)

# Mnist digits dataset
train_data = torchvision.datasets.MNIST(
    root='data/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)

test_data = torchvision.datasets.MNIST(
    root='data/',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)

train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

CUDA available? True


创建网络

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        return out

net=Net()

# 将模型的参数送到GPU中
if GPU_FLAG==True:
    net.cuda()
print(net) # 输出模型结构

criterion = nn.CrossEntropyLoss() # 定义loss函数
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

Net (
  (fc1): Linear (784 -> 256)
  (fc2): Linear (256 -> 10)
)


检验 estimaton

In [5]:
def evaluate():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.view(-1, 28 * 28)).cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum()
    return 1.0 * correct / total

training

In [6]:
for i in xrange(1,1+training_epoch):
    cost = 0.0
    for j, (images, labels) in enumerate(train_loader):
        if GPU_FLAG:
            x = Variable(images).view(-1, 28 ** 2).cuda()
            y = Variable(labels).cuda()
        else:
            x = Variable(images)
            y = Variable(labels)
            
        optimizer.zero_grad()
        y_ = net(x)
        loss = criterion(y_, y)
        loss.backward()
        optimizer.step()

        cost += loss.data[0]
    cost /= j + 1
    if i % 10 == 0:
        print('Epoch %s / %s, training loss: %s, test accuracy: %s' % (i, training_epoch, cost, evaluate()))

Epoch 10 / 100, training loss: 0.0180823849274, test accuracy: 0.996883333333
Epoch 20 / 100, training loss: 0.0031534965643, test accuracy: 0.99855
Epoch 30 / 100, training loss: 0.000151748574426, test accuracy: 0.999716666667
Epoch 40 / 100, training loss: 6.16827692168e-05, test accuracy: 1.0
Epoch 50 / 100, training loss: 4.14097788522e-05, test accuracy: 1.0
Epoch 60 / 100, training loss: 5.09822486252e-05, test accuracy: 1.0
Epoch 70 / 100, training loss: 1.0547668332e-05, test accuracy: 1.0
Epoch 80 / 100, training loss: 1.51084291302e-05, test accuracy: 1.0
Epoch 90 / 100, training loss: 0.000142188140945, test accuracy: 1.0
Epoch 100 / 100, training loss: 5.8979481126e-06, test accuracy: 1.0
